# 大陸某產品銷量分析

# 引入數據

In [1]:
import pandas as pd
path = 'C:/Users/abcd8/OneDrive/桌面/resume/project2'

In [2]:
df1 = pd.read_csv(f'{path}/(sample)sam_tianchi_mum_baby_trade_history.csv')
df2 = pd.read_csv(f'{path}/(sample)sam_tianchi_mum_baby.csv')

In [3]:
df1.head(5)

,user_id,auction_id,cat_id,cat1,property,buy_mount,day
0,786295544,41098319944,50014866,50022520,21458:86755362;13023209:3593274;10984217:21985...,2,20140919
1,532110457,17916191097,50011993,28,21458:11399317;1628862:3251296;21475:137325;16...,1,20131011
2,249013725,21896936223,50012461,50014815,21458:30992;1628665:92012;1628665:3233938;1628...,1,20131011
3,917056007,12515996043,50018831,50014815,21458:15841995;21956:3494076;27000458:59723383...,2,20141023
4,444069173,20487688075,50013636,50008168,21458:30992;13658074:3323064;1628665:3233941;1...,1,20141103


In [4]:
df2.head(5)

,user_id,birthday,gender
0,2757,20130311,1
1,415971,20121111,0
2,1372572,20120130,1
3,10339332,20110910,0
4,10642245,20130213,0


# 將兩個檔案合併，並以 auction_id 去除重複列

In [5]:
df = pd.merge(df1, df2, on='user_id',how='right')
df['day'] = pd.to_datetime(df['day'], format='%Y%m%d')
df.head(2)

,user_id,auction_id,cat_id,cat1,property,buy_mount,day,birthday,gender
0,2757,17429550751,50010555,50008168,21458:30992;25935:31381;1628665:29796;1628665:...,1,2013-04-10,20130311,1
1,415971,20854308837,50010548,50008168,1628665:131622;25935:21991;22019:31001;22019:3...,1,2013-01-28,20121111,0


In [6]:
df = df.drop_duplicates('auction_id')
df.head(2)

,user_id,auction_id,cat_id,cat1,property,buy_mount,day,birthday,gender
0,2757,17429550751,50010555,50008168,21458:30992;25935:31381;1628665:29796;1628665:...,1,2013-04-10,20130311,1
1,415971,20854308837,50010548,50008168,1628665:131622;25935:21991;22019:31001;22019:3...,1,2013-01-28,20121111,0


In [7]:
from pyecharts import options as opts
from pyecharts.charts import Line

# 以日期與銷量進行計算

In [9]:
sales_by_day = df.groupby('day')['buy_mount'].sum().reset_index()

In [10]:
sales_by_day.head(2)

,day,buy_mount
0,2012-07-02,6
1,2012-07-05,1


In [11]:
sales_by_day.sort_values(by='buy_mount',ascending =False)

,day,buy_mount
55,2012-11-05,161
323,2014-02-18,43
150,2013-05-03,30
393,2014-05-21,27
154,2013-05-11,26
...,...,...
341,2014-03-16,1
340,2014-03-15,1
339,2014-03-14,1
338,2014-03-13,1


# 繪製時間與銷量折線圖

In [12]:
from pyecharts.charts import Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType
line = (
    Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS))
    .add_xaxis(list(sales_by_day['day']))
    .add_yaxis('銷量',list(sales_by_day['buy_mount']))
    .set_global_opts(title_opts=opts.TitleOpts(title="時間與銷量變化圖",subtitle=""))
)
line.render_notebook()

# 每年有部分月份銷量明顯增加，以下針對每年每月份進行銷量計算

In [13]:
a = sales_by_day.groupby([sales_by_day['day'].dt.year, sales_by_day['day'].dt.month])['buy_mount'].sum().unstack()
a

day,1,2,3,4,5,6,7,8,9,10,11,12
day,,,,,,,,,,,,
2012,NaN,NaN,NaN,NaN,NaN,NaN,24.0,27.0,24.0,30.0,196.0,20.0
2013,55.0,15.0,33.0,32.0,94.0,24.0,25.0,32.0,45.0,42.0,56.0,49.0
2014,34.0,68.0,58.0,47.0,70.0,57.0,47.0,55.0,45.0,48.0,60.0,54.0
2015,54.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 繪製每年每月銷量折線圖

In [15]:
import pyecharts.options as opts
from pyecharts.charts import Line

x_data = ['一月','二月','三月','四月','五月','六月','七月','八月','九月','十月','十一月','十二月']
# y_data = list(a.iloc[0])

(
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="2012",
        stack="总量",
        y_axis=list(a.iloc[0]),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="2013",
        stack="总量",
        y_axis=list(a.iloc[1]),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="2014",
        stack="总量",
        y_axis=list(a.iloc[2]),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="2015",
        stack="总量",
        y_axis=list(a.iloc[3]),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各年每月銷量"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
    )
    .render_notebook()
)


# 可以看出每年的十一月銷量明顯增加

# cat1為品項，以下針對全部年份各品項的銷售總量進行計算

In [18]:
df.cat1.unique()

array([ 50008168,        28,  50014815,        38, 122650008,  50022520],
      dtype=int64)

In [19]:
df.head(5)

,user_id,auction_id,cat_id,cat1,property,buy_mount,day,birthday,gender
0,2757,17429550751,50010555,50008168,21458:30992;25935:31381;1628665:29796;1628665:...,1,2013-04-10,20130311,1
1,415971,20854308837,50010548,50008168,1628665:131622;25935:21991;22019:31001;22019:3...,1,2013-01-28,20121111,0
2,1372572,16915013171,50008845,28,21458:30992;1628665:3233941;1628665:3233942;16...,1,2013-03-27,20120130,1
3,10339332,13174075495,50001732,50014815,21458:3409452;3066697:92335415;2815901:9233541...,1,2014-05-26,20110910,0
4,10642245,14109039851,50006843,38,21458:7142737;8694098:95303334;12786373:54223;...,1,2013-06-17,20130213,0


In [20]:
data = df.groupby('cat1')['buy_mount'].sum()

In [21]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

test = data
test.index = test.index.astype(str)



c = (
    Pie()
    .add("銷量", [list(z) for z in zip(test.index, data)])
    .set_colors(["blue", "green", "yellow", "red", "pink", "orange"])
    .set_global_opts(title_opts=opts.TitleOpts(title="各項銷量"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render_notebook()
)
c

# 計算各年度各品項的銷量

In [25]:
years_data = df.groupby([df['day'].dt.year,'cat1'])['buy_mount'].sum()
years_data

day   cat1     
2012  28            23
      38            25
      50008168      66
      50014815     184
      50022520      18
      122650008      5
2013  28           129
      38            58
      50008168     139
      50014815     129
      50022520      26
      122650008     21
2014  28           131
      38            63
      50008168     225
      50014815     154
      50022520      45
      122650008     25
2015  28            13
      38             2
      50008168      20
      50014815      20
      50022520       2
      122650008      3
Name: buy_mount, dtype: int64

# 繪製長條圖

In [67]:
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType


index = ['28', '38', '50008168', '50014815', '50022520', '122650008']

d = (
    Bar({"theme": ThemeType.MACARONS})
    .add_xaxis(index)
    .add_yaxis("2012", list(years_data.loc[(2012)]))
    .add_yaxis("2013", list(years_data.loc[(2013)]))
    .add_yaxis("2014", list(years_data.loc[(2014)]))
    .add_yaxis("2015", list(years_data.loc[(2015)]))
    .set_global_opts(
        title_opts={"text": "各產品不同年度銷售量", "subtext": "銷量"}
    )
    .render_notebook()
)
d

# 2015年度各產品銷量明顯降低